In [ ]:
# Connect google drive account
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Imports
!pip install pycountry

import pycountry
import os
import pandas as pd
import numpy as np
import spacy
from spacy.matcher import PhraseMatcher

In [ ]:
# Import files
file1 = "/content/drive/My Drive/gdelt_protests_2018_2021/protests_1.csv"
file2 = "/content/drive/My Drive/gdelt_protests_2018_2021/protests_1.csv"

df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Compare counts before concatenation
len1 = df1.shape[0]
len2 = df2.shape[0]
print(f"File1: {len1} rows, File2: {len2} rows, Combined: {len1 + len2}")

File1: 4932955 rows, File2: 4932955 rows, Combined: 9865910


In [ ]:
# Concat datasets
df = pd.concat([df1, df2], ignore_index = True)

In [ ]:
# Check for duplicates
# before = len(df)
# df = df.drop_duplicates()
# after = len(df)
# print(f"Dropped {before-after} exact duplicates")

# Drop duplicates based on key columns
key_cols = ['SQLDATE','ActionGeo_Lat','ActionGeo_Long','Actor1Name','Actor2Name'
,'EventCode']
unique_before = df.shape[0]
df = df.drop_duplicates(subset=key_cols)
print(f"Dropped {unique_before - df.shape[0]} duplicates based on {key_cols}")

Dropped 5433377 duplicates based on ['SQLDATE', 'ActionGeo_Lat', 'ActionGeo_Long', 'Actor1Name', 'Actor2Name', 'EventCode']


In [ ]:
# Convert Data Format
df['SQLDATE'] = pd.to_datetime(df['SQLDATE'], format='%Y%m%d', errors='coerce')


In [ ]:
# Drop rows with missing data
df = df.dropna(subset=['SQLDATE', 'ActionGeo_Lat', 'ActionGeo_Long', 'AvgTone']).copy()


# Round data to make charts easier to read
df['AvgTone'] = df['AvgTone'].round(2)
df['GoldsteinScale'] = df['GoldsteinScale'].round(2)


df.head()

,SQLDATE,Actor1Name,Actor2Name,EventRootCode,EventCode,GoldsteinScale,AvgTone,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long
0,2018-06-12,NaN,BOLIVIA,14,140,-6.5,-0.47,BL,-17.000000,-65.0
1,2018-10-18,NaN,GUINEA BISSAU,14,141,-6.5,0.21,GV,11.000000,-10.0
2,2018-12-21,NaN,EMPLOYEE,14,140,-6.5,-6.28,HU,47.000000,20.0
3,2018-10-10,NaN,SOUTH KOREA,14,141,-6.5,-3.59,KS,37.000000,127.5
4,2018-07-16,NaN,PARLIAMENT,14,141,-6.5,-0.24,SN,1.366667,103.8


In [ ]:
# Replace missing actors with unknown, and lowercase all actor names
df['Actor1Name'] = df['Actor1Name'].fillna('Unknown Actor 1')
df['Actor2Name'] = df['Actor2Name'].fillna('Unknown Actor 2')

df.head()

,SQLDATE,Actor1Name,Actor2Name,EventRootCode,EventCode,GoldsteinScale,AvgTone,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long
0,2018-06-12,Unknown Actor 1,BOLIVIA,14,140,-6.5,-0.47,BL,-17.000000,-65.0
1,2018-10-18,Unknown Actor 1,GUINEA BISSAU,14,141,-6.5,0.21,GV,11.000000,-10.0
2,2018-12-21,Unknown Actor 1,EMPLOYEE,14,140,-6.5,-6.28,HU,47.000000,20.0
3,2018-10-10,Unknown Actor 1,SOUTH KOREA,14,141,-6.5,-3.59,KS,37.000000,127.5
4,2018-07-16,Unknown Actor 1,PARLIAMENT,14,141,-6.5,-0.24,SN,1.366667,103.8


In [ ]:
# Create a new column to see if the protest was pre-COVID (3/1/2020)

df['COVID_Era'] = np.where(df['SQLDATE'] < '2020-03-01', 'Pre-COVID',
                           'COVID-Era')

In [ ]:
# Track motivations of the protest using the Event Code

# Convert EventCode to string if it's numeric
df['EventCode'] = df['EventCode'].astype(str)

# Define conditions and corresponding motivations
conditions = [
    df['EventCode'] == '141',
    df['EventCode'] == '142',
    df['EventCode'] == '143',
    df['EventCode'] == '144',
    df['EventCode'] == '145'
]

motivations = [
    'Policy Change',
    'Anti-Government',
    'Anti-Business',
    'Group Rights',
    'Anti-Discrimination'
]

# Default fallback if no match
df['ProtestMotivation'] = np.select(conditions, motivations,
                                    default='General Protest')


In [ ]:
# Check the results
df.head(10)

,SQLDATE,Actor1Name,Actor2Name,EventRootCode,EventCode,GoldsteinScale,AvgTone,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long,COVID_Era,ProtestMotivation
0,2018-06-12,Unknown Actor 1,BOLIVIA,14,140,-6.5,-0.47,BL,-17.000000,-65.000000,Pre-COVID,General Protest
1,2018-10-18,Unknown Actor 1,GUINEA BISSAU,14,141,-6.5,0.21,GV,11.000000,-10.000000,Pre-COVID,Policy Change
2,2018-12-21,Unknown Actor 1,EMPLOYEE,14,140,-6.5,-6.28,HU,47.000000,20.000000,Pre-COVID,General Protest
3,2018-10-10,Unknown Actor 1,SOUTH KOREA,14,141,-6.5,-3.59,KS,37.000000,127.500000,Pre-COVID,Policy Change
4,2018-07-16,Unknown Actor 1,PARLIAMENT,14,141,-6.5,-0.24,SN,1.366667,103.800000,Pre-COVID,Policy Change
5,2019-07-15,Unknown Actor 1,MINIST OF SECURITY,14,141,-6.5,-3.76,TO,8.000000,1.166667,Pre-COVID,Policy Change
6,2019-02-16,Unknown Actor 1,PRESIDENT,14,140,-6.5,-6.82,HA,18.539200,-72.335000,Pre-COVID,General Protest
7,2019-05-01,Unknown Actor 1,MACEDONIA,14,141,-6.5,-5.26,MK,42.000000,21.433300,Pre-COVID,Policy Change
8,2018-12-25,Unknown Actor 1,POLICE,14,141,-6.5,-5.81,BK,43.906100,18.320800,Pre-COVID,Policy Change
9,2018-11-25,Unknown Actor 1,PARLIAMENT,14,141,-6.5,-9.22,AL,41.231400,20.156100,Pre-COVID,Policy Change


Some actor2Names are countries, which makes python string matching inefficient. We may need to use NER to improve accuracy. We can get a list of unique actor names and see what are the patterns and outliers.

In [ ]:
# Get unique actor names
actor2_unique = df['Actor2Name'].dropna().str.upper().value_counts().head(100)
actor2_unique.to_frame().reset_index().rename(columns={'index': 'Actor2Name',
                                                       'Actor2Name': 'Count'})


,Count,count
0,UNKNOWN ACTOR 2,1507277
1,UNITED STATES,143992
2,POLICE,131164
3,GOVERNMENT,124922
4,PRESIDENT,69055
...,...,...
95,MIGRANT,5484
96,NICARAGUA,5456
97,IRELAND,5401
98,VIETNAM,5398


In [ ]:
# 1) Load spaCy and build a PhraseMatcher
nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")

patterns = {
    "Civilians":           ["protester","demonstrator","student","worker",
                            "citizen","residents","village","employee","ukrainian"],
    "Government":          ["police", "republic", "kingdom","governor","regime",
                            "parliament","army","military","government",
                            "security","state","president","authorities",
                            "authority","prime minister","chancellor",
                            "congress","legislature","court","judiciary",
                            "the white house","russia","venezuela","ukraine"],
    "Political Party":     ["party","minister","candidate","politician",
                            "congressman","congresswoman"],
    "NGO / Advocacy":      ["ngo","nonprofit","human rights","activist",
                            "charity","organization"],
    "Corporate / Business":["company","corporation","bank","industry","firm",
                            "business","companies"],
    "Agriculture":         ["farm","farmer","agriculture","landowner"],
    "Healthcare":          ["hospital","medical","healthcare","nurse","doctor"],
    "Prison Reform":       ["prison","incarceration","inmate","detention"],
    "Media Reform":        ["media","press","journalist","news agency"],
    "Religious":           ["christian","muslim","hindu","jewish","buddhism","jain"]
}

for label, terms in patterns.items():
    matcher.add(label, [nlp.make_doc(t) for t in terms])

# 2) Seed every ISO country name into Government
country_docs = [nlp.make_doc(c.name) for c in pycountry.countries]
matcher.add("Government", country_docs)

# 3) Deduplicate and build classification map
all_actors = pd.concat([df['Actor1Name'], df['Actor2Name']]).dropna().unique()
name_to_cat = {}
name_to_ner = {}

for raw_name in all_actors:
    # Title-case for NER, but keep matcher case‑insensitive
    check_text = raw_name if raw_name and raw_name[0].isupper()
    else raw_name.title()
    doc = nlp(check_text)

    # 3a) capture raw NER label
    name_to_ner[raw_name] = doc.ents[0].label_ if doc.ents else "Unknown"

    # 3b) PhraseMatcher first
    m = matcher(doc)
    if m:
        name_to_cat[raw_name] = nlp.vocab.strings[m[0][0]]
        continue

    # 3c) NER fallback on properly-cased doc
    if doc.ents:
        ent = doc.ents[0].label_
        if ent in ("GPE","LOC"):
            name_to_cat[raw_name] = "Government"
        elif ent == "ORG":
            name_to_cat[raw_name] = "NGO / Advocacy"
        elif ent == "NORP":
            low = raw_name.lower()
            if any(r in low for r in patterns["Religious"]):
                name_to_cat[raw_name] = "Religious"
            else:
                name_to_cat[raw_name] = "Civilians"
        else:
            name_to_cat[raw_name] = "Unknown"
    else:
        name_to_cat[raw_name] = "Unknown"

# 4) Map back to data frame
df['Actor1_NER'] = np.vectorize(name_to_ner.get)(df['Actor1Name'].values)
df['Actor2_NER'] = np.vectorize(name_to_ner.get)(df['Actor2Name'].values)
df['PrimaryActorType'] = np.vectorize(name_to_cat.get)(df['Actor1Name'].values)
df['SecondaryActorType'] = np.vectorize(name_to_cat.get)(df['Actor2Name'].values)

# 5) Anti‑government override
df.loc[df['EventRootCode'].astype(str)=='142', 'SecondaryActorType'] = 'Government'

# 6) convert remaining GPE/LOC in Actor1 to be Civilians
df.loc[df['Actor1_NER'].isin(['GPE','LOC']), 'PrimaryActorType'] = 'Civilians'

According to GDELT's CAMEO naming convention, Actor1 protests 'against' Actor2. But, in the above nationalities in actor2 are misclassified as civillian, when they are more likely to be the government which is being protested against. We need to override the secondary actor type here, and ensure that if a country's denonoym appears as Actor2, the category is 'government' and not civillian.

In the same way, Actor1 may be misclassified as government when it hits the matcher, although we know Actor1 are the protestors.

In [ ]:
# Check changes
df[['Actor2Name','SecondaryActorType','ProtestMotivation']].sample(10)

,Actor2Name,SecondaryActorType,ProtestMotivation
1039789,Unknown Actor 2,Unknown,Anti-Discrimination
4443083,Unknown Actor 2,Unknown,Policy Change
1853166,Unknown Actor 2,Unknown,Policy Change
92643,CUBA,Government,General Protest
4730159,FREE STATE,Government,Policy Change
4605110,UNITED STATES,Government,Policy Change
2842657,Unknown Actor 2,Unknown,Policy Change
1888213,NORWAY,Government,Policy Change
2843163,FRANCE,Government,Anti-Discrimination
2999845,Unknown Actor 2,Unknown,Policy Change


In [ ]:
# Check the count of unknown actors in comparison to correctly categorized
uncategorized_actor_1 = df[df['PrimaryActorType'] == 'Unknown']['Actor1Name'].count()
uncategorized_actor_2 = df[df['SecondaryActorType'] == 'Unknown']['Actor2Name'].count()

print(f"Unknown actors in Actor1: {uncategorized_actor_1}")
print(f"Unknown actors in Actor2: {uncategorized_actor_2}")

total_actors = len(df)
print(f"Total actors: {total_actors}")

percent_uknown_actors_1 = (uncategorized_actor_1) / total_actors * 100
percent_uknown_actors_2 = (uncategorized_actor_2) / total_actors * 100

print(f"Percentage of unknown primary actors: {percent_uknown_actors_1:.2f}%")
print(f"Percentage of unknown secondary actors: {percent_uknown_actors_2:.2f}%")

Unknown actors in Actor1: 965095
Unknown actors in Actor2: 1862509
Total actors: 4205262
Percentage of unknown primary actors: 22.95%
Percentage of unknown secondary actors: 44.29%


The improvements to the matcher have reduced the unknown actors marginally. However, because secondary actors, AKA targets of the protests, can be individually people, as well as geographic areas smaller than countries it is harder to significantly increase coverage.

In [ ]:
# Make columns more reader-friendly
df = df.rename(columns={
    'SQLDATE':            'Date',
    'Actor1Name':         'Primary Actor',
    'Actor2Name':         'Secondary Actor',
    'EventRootCode':      'Root Code',
    'EventCode':          'Event Code',
    'GoldsteinScale':     'Goldstein Scale',
    'AvgTone':            'Average Tone',
    'ActionGeo_CountryCode':'Country Code',
    'ActionGeo_Lat':      'Latitude',
    'ActionGeo_Long':     'Longitude',
    'PrimaryActorType':   'Primary Actor Type',
    'SecondaryActorType': 'Secondary Actor Type',
    'ProtestMotivation':  'Motivation',
    'COVID_Era':          'Era'
})


In [ ]:
# Separate date into parts
df['Year']  = df['Date'].dt.year
df['Month'] = df['Date'].dt.month_name().str.slice(stop=3)
df['MonthNum'] = df['Date'].dt.month

In [ ]:
# Filter out coordinates that don't make sense (outside of logical bounds)
# Filter out placeholder coordinates (0.0, 0.0) where location is unknown
df = df[
  df['Latitude'].between(-90, 90) &
  df['Longitude'].between(-180, 180) &
  ~((df['Latitude'] == 0) & (df['Longitude'] == 0))
]


In [ ]:
print("Final events:", df.shape[0])

Final events: 4205262


In [ ]:
# Check primary actor type and covid era
print(df.groupby('Primary Actor Type').size().sort_values(ascending=False).head())
print(df.groupby(['Year','Era']).size())


Primary Actor Type
Civilians               1229368
Government              1021191
Unknown                  965095
NGO / Advocacy           776953
Corporate / Business      71658
dtype: int64
Year  Era      
2017  Pre-COVID    901593
2018  Pre-COVID    815661
2019  Pre-COVID    784171
2020  COVID-Era    577808
      Pre-COVID    117940
2021  COVID-Era    544451
2022  COVID-Era    463638
dtype: int64


This looks more correct, the matcher has correctly categorized more civillian protests than goverment

In [ ]:
# Convert country codes to country

fips_url = "https://www.gdeltproject.org/data/lookups/FIPS.country.txt"
fips = pd.read_csv(
    fips_url,
    sep="\t",
    header=None,
    names=["Country Code","Country"],  # first column is code, second is name
    dtype=str
)

# 3) Merge on Country Code column
df = df.merge(fips, on="Country Code", how="left")

# 4) Check how many still unmapped
missing = df["Country"].isna().sum()
print(f"Rows without a country name: {missing:,}")

Rows without a country name: 6,545


In [ ]:
df.head()

,Date,Primary Actor,Secondary Actor,Root Code,Event Code,Goldstein Scale,Average Tone,Country Code,Latitude,Longitude,Era,Motivation,Primary Actor Type,Secondary Actor Type,Actor1_NER,Actor2_NER,Year,Month,MonthNum,Country
0,2018-06-12,Unknown Actor 1,BOLIVIA,14,140,-6.5,-0.47,BL,-17.000000,-65.0,Pre-COVID,General Protest,Unknown,Unknown,CARDINAL,PERSON,2018,Jun,6,Bolivia
1,2018-10-18,Unknown Actor 1,GUINEA BISSAU,14,141,-6.5,0.21,GV,11.000000,-10.0,Pre-COVID,Policy Change,Unknown,Government,CARDINAL,Unknown,2018,Oct,10,Equatorial Guinea
2,2018-12-21,Unknown Actor 1,EMPLOYEE,14,140,-6.5,-6.28,HU,47.000000,20.0,Pre-COVID,General Protest,Unknown,Civilians,CARDINAL,Unknown,2018,Dec,12,Hungary
3,2018-10-10,Unknown Actor 1,SOUTH KOREA,14,141,-6.5,-3.59,KS,37.000000,127.5,Pre-COVID,Policy Change,Unknown,Government,CARDINAL,GPE,2018,Oct,10,South Korea
4,2018-07-16,Unknown Actor 1,PARLIAMENT,14,141,-6.5,-0.24,SN,1.366667,103.8,Pre-COVID,Policy Change,Unknown,Government,CARDINAL,ORG,2018,Jul,7,Singapore


In [ ]:
# List the unique codes that didn’t merge
missing_codes = (
    df.loc[df['Country'].isna(), 'Country Code']
      .value_counts()
      .reset_index()
      .rename(columns={'index':'Country Code','Country Code':'Count'})
)
print(missing_codes)


  Count  count
0    RB   6340
1    OC    205


In [ ]:
# See sample of records where country is missing for RB
# Show the first 5 rows where Country Code == 'RB'
df.loc[df['Country Code'] == 'RB'].head(5)

,Date,Primary Actor,Secondary Actor,Root Code,Event Code,Goldstein Scale,Average Tone,Country Code,Latitude,Longitude,Era,Motivation,Primary Actor Type,Secondary Actor Type,Actor1_NER,Actor2_NER,Year,Month,MonthNum,Country
404,2018-04-19,SERBIA,CROATIA,14,145,-7.5,-4.81,RB,44.8186,20.4681,Pre-COVID,Anti-Discrimination,Government,Government,Unknown,Unknown,2018,Apr,4,NaN
576,2020-07-10,SERBS,GOVERNMENT,14,141,-6.5,-12.79,RB,44.8186,20.4681,COVID-Era,Policy Change,NGO / Advocacy,Government,ORG,Unknown,2020,Jul,7,NaN
577,2020-10-30,GOVERNMENT,Unknown Actor 2,14,141,-6.5,-4.90,RB,44.8186,20.4681,COVID-Era,Policy Change,Government,Unknown,Unknown,CARDINAL,2020,Oct,10,NaN
690,2019-01-18,Unknown Actor 1,RUSSIA,14,141,-6.5,1.26,RB,44.8186,20.4681,Pre-COVID,Policy Change,Unknown,Government,CARDINAL,Unknown,2019,Jan,1,NaN
771,2019-05-08,CHINA,EMBASSY,14,141,-6.5,-7.01,RB,44.8186,20.4681,Pre-COVID,Policy Change,Civilians,NGO / Advocacy,GPE,ORG,2019,May,5,NaN


The percentage of unknown records is really small, I think we can either leave them or remove them from the dataset.

In [ ]:
# Save final data frame as BigQuery dataset
from google.colab import auth
auth.authenticate_user()


In [ ]:
from google.cloud import bigquery

# Initialize client
project_id = 'gdelt-protests-2019-2022'
client     = bigquery.Client(project=project_id)
print("Using project:", client.project)

# Create new dataset
dataset_id = f"{project_id}.gdelt_analysis"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
client.create_dataset(dataset, exists_ok=True)
print("Dataset ready:", dataset_id)


Using project: gdelt-protests-2019-2022
Dataset ready: gdelt-protests-2019-2022.gdelt_analysis


In [ ]:
# Load dataframe to BigQuery
table_id = f"{dataset_id}.protests_2017_2022"
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE", autodetect=True)
load_job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
load_job.result()

print("Loaded rows:", client.get_table(table_id).num_rows)


Loaded rows: 4205262


In [1]:
# Save final to Drive
output_path = '/content/drive/My Drive/gdelt_protests_2018_2021/gdelt_protests_final_2017_2022.csv'
df.to_csv(output_path, index=False)
print(" Saved to", output_path)

NameError: name 'df' is not defined